In [37]:
import json
from myutils.func_decorated import log_func_time
input_dict = json.load(open('data/container_info.json', "rb")) # 剩下的就是解析了，都是列表和字典的操作
container_b20 = input_dict['-20']
# container_b20.remove('温度使用范围_C')
one_order_str = '107==>12*8*2;'.rstrip(';')
quantity = int(one_order_str.split('==>')[0])
goods_shape = one_order_str.split('==>')[1]
[goods_l,goods_w,goods_h] = [int(i) for i in goods_shape.split('*')]
print([goods_l,goods_w,goods_h])
goods_vol_L = round(quantity*goods_l*goods_w*goods_h/1000,2)
print(goods_vol_L)

######## 查看哪些箱子能放得下
for k,v in container_b20.items():
    if v['最大容积_L']>goods_vol_L:
        v.pop('温度使用范围_C') 
        print(k, '===>', v)

[12, 8, 2]
20.54
EP53 ===> {'保温箱内径_mm': '425*385*335', '长': 425, '宽': 385, '高': 335, '有效高': 185, '成本': 2.85, '最大容积_L': 24.67}
EP133 ===> {'保温箱内径_mm': '660*490*395', '长': 660, '宽': 490, '高': 395, '有效高': 245, '成本': 4.08, '最大容积_L': 57.48}
EP46 ===> {'保温箱内径_mm': '435*380*280', '长': 435, '宽': 380, '高': 280, '有效高': 130, '成本': 2.64, '最大容积_L': 20.83}
EP78 ===> {'保温箱内径_mm': '506*406*376', '长': 506, '宽': 406, '高': 376, '有效高': 226, '成本': 3.27, '最大容积_L': 38.62}
EP125 ===> {'保温箱内径_mm': '506*406*610', '长': 506, '宽': 406, '高': 610, '有效高': 460, '成本': 4.04, '最大容积_L': 62.66}


In [42]:
@log_func_time
def one_goods_one_box_package(box_name, goods_str):
    print('=====================================')
    mydict = {}
    one_order_str = goods_str.rstrip(';')
    quantity = int(one_order_str.split('==>')[0])
    goods_shape = one_order_str.split('==>')[1]
    [goods_l,goods_w,goods_h] = [int(i)*10 for i in goods_shape.split('*')]
    print('数量',quantity, '尺寸',[goods_l,goods_w,goods_h], '单位：cm')
    box_dict = container_b20.get(box_name)
#     print('container信息：', box_dict)
    
    ########===================== 不可倒放的情况下，有两种放法
    mydict['最大层数'] = box_dict.get('有效高')//(goods_h)
    
    mydict['长长数'] = box_dict.get('长')//goods_l  # 货物的长对应container的长
    mydict['宽宽数'] = box_dict.get('宽')//goods_w
    mydict['宽长数'] = box_dict.get('宽')//goods_l
    mydict['长宽数'] = box_dict.get('长')//goods_w
    
    mydict['长长结余量']= box_dict.get('长')%goods_l  # 长长放法长度还剩余的量
    mydict['宽宽结余量']= box_dict.get('宽')%goods_w
    mydict['长宽结余量']= box_dict.get('长')%goods_w
    mydict['宽长结余量']= box_dict.get('宽')%goods_l
    
    #  长长放法长度剩余量，剩余的container长度可以放几个货宽
    mydict['长长结余量宽数']=mydict['长长结余量']//goods_w  # 长长放法长度剩余量，可放宽数
    mydict['最终长结余']=mydict['长长结余量']%goods_w  # 长长放法长度剩余量，可放宽数
    mydict['长长结余']=[mydict['最终长结余'], mydict['宽宽结余量']]
    
    #  宽长放法宽度剩余量，剩余的container宽度可以放几个货宽
    mydict['宽长结余量宽数']=mydict['宽长结余量']//goods_l 
    mydict['最终宽结余']=mydict['宽长结余量']%goods_l
    mydict['长宽结余']=[mydict['长宽结余量'], mydict['最终宽结余']]
    
    ## 长长放法
    mydict['长长放法-层最大数量'] = mydict['长长数']*mydict['宽宽数']+mydict['宽长数']*mydict['长长结余量宽数']
    max_h_v_num =  (box_dict.get('最大容积_L')*1000000/(mydict['长长放法-层最大数量']*goods_l*goods_w))//goods_h # 从体积角度出发的最大高度
    max_num = min(max_h_v_num, mydict['最大层数'])
    print('长长---从体积和高度角度出发的最大层数', [max_h_v_num, mydict['最大层数']])
    mydict['长长放法-体最大数量'] = max_num*mydict['长长放法-层最大数量'] 
    
    ## 长宽放法
    mydict['长宽放法-层最大数量'] = mydict['长宽数']*mydict['宽长数']+mydict['长长数']*mydict['宽长结余量宽数']
    max_h_v_num2 =  (box_dict.get('最大容积_L')*1000000/(mydict['长宽放法-层最大数量']*goods_l*goods_w))//goods_h # 从体积角度出发的最大高度
    max_num2 = min(max_h_v_num2, mydict['最大层数'])
    print('长宽---从体积和高度角度出发的最大层数', [max_h_v_num2, mydict['最大层数']])
    mydict['长宽放法-体最大数量'] = max_num2*mydict['长宽放法-层最大数量'] 
#     for k,v in mydict.items():
#         print(k, '===>', v)
    
    print('长长放法,长宽放法,最大数量对比')
    print(mydict['长长放法-体最大数量'], mydict['长宽放法-体最大数量'],
          '长宽结余量===>',mydict['长长结余'],mydict['长宽结余'])
    print('container信息：', box_dict)
    return mydict['长长放法-体最大数量'], mydict['长宽放法-体最大数量']

    
res1 = one_goods_one_box_package('EP53', '107==>12*8*2;')
########## 假设可以倒放
res2 = one_goods_one_box_package('EP53', '107==>2*8*12;')  # 长高互换
res3 = one_goods_one_box_package('EP53', '107==>12*2*8;')  # 宽高互换


数量 107 尺寸 [120, 80, 20] 单位：cm
长长---从体积和高度角度出发的最大层数 [10.0, 9]
长宽---从体积和高度角度出发的最大层数 [8.0, 9]
长长放法,长宽放法,最大数量对比
108 120.0 长宽结余量===> [65, 65] [25, 25]
container信息： {'保温箱内径_mm': '425*385*335', '长': 425, '宽': 385, '高': 335, '有效高': 185, '成本': 2.85, '最大容积_L': 24.67}
func time spend info: [func: one_goods_one_box_package] [cost: 0.000 ms] [args: ('EP53', '107==>12*8*2;')]  [kwargs: {}]
数量 107 尺寸 [20, 80, 120] 单位：cm
长长---从体积和高度角度出发的最大层数 [1.0, 1]
长宽---从体积和高度角度出发的最大层数 [1.0, 1]
长长放法,长宽放法,最大数量对比
84.0 95.0 长宽结余量===> [5, 65] [25, 5]
container信息： {'保温箱内径_mm': '425*385*335', '长': 425, '宽': 385, '高': 335, '有效高': 185, '成本': 2.85, '最大容积_L': 24.67}
func time spend info: [func: one_goods_one_box_package] [cost: 0.000 ms] [args: ('EP53', '107==>2*8*12;')]  [kwargs: {}]
数量 107 尺寸 [120, 20, 80] 单位：cm
长长---从体积和高度角度出发的最大层数 [1.0, 2]
长宽---从体积和高度角度出发的最大层数 [2.0, 2]
长长放法,长宽放法,最大数量对比
66.0 126.0 长宽结余量===> [5, 5] [5, 25]
container信息： {'保温箱内径_mm': '425*385*335', '长': 425, '宽': 385, '高': 335, '有效高': 185, '成本': 2.85, '最大容积_L

In [43]:
@log_func_time
def one_goods_multi_box_package(goods_str):
    '''
    比较简单，重点考虑箱子怎么选，是选两个中的还是一大一小？
    '''
    # pass

In [ ]:
@log_func_time
def multi_goods_one_box_package(goods_str):
    '''
    ### 先放什么后放什么？
    '''
    # pass

In [ ]:
@log_func_time
def multi_goods_two_type_box_and_one_type_box_fix_package(goods_str):
    '''
    ### 多物品并且其中一种类型箱子固定
    '''
    # pass